In [3]:
import pickle
from pylatex import Document, Section, Enumerate
from pylatex.utils import italic, bold, dumps_list, NoEscape
import itertools
import random

In [16]:
def read_data_iterator(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        for i in itertools.count():
            try:
                yield pickle.load(f)
            except EOFError:
                return

In [18]:
data = list(read_data_iterator('dataset/extractions.pickle'))
random.seed('event_chain_1')
random.shuffle(data)
data[0]['entity']

'Holmes'

In [20]:
def create_pdf(data, pdf_name, use_triples=False, answer_key=False):
    text_key = ('triples' if use_triples else 'sentences')
    doc = Document()
    with doc.create(Enumerate()) as question_enum:
        for i, seq in zip(range(20), data):
            if use_triples:
                question_enum.add_item(
                    dumps_list((
                       NoEscape("(%s)" % dumps_list((italic(seq['entity']) if t is None else t for t in triple), token=', ')) 
                       for triple in seq['triples']
                    ), 
                        token=', ')
                )
            else:
                question_enum.add_item(' '.join([s.strip() for s in seq['sentences']]))
            with doc.create(Enumerate(enumeration_symbol=r"\alph*)")) as candidate_enum:
                for i, candidate  in enumerate(seq['candidates']):
                    lat_str = dumps_list((italic(seq['entity']) if c is None else c for c in candidate), escape=False, token=', ')
                    lat_str = NoEscape("(%s)"%lat_str)
                    if answer_key and i == seq['correct']:
                        lat_str = bold(lat_str)
                    candidate_enum.add_item(lat_str)

    doc.generate_pdf(pdf_name, silent=True, clean_tex=True)
    
create_pdf(data[:20], 'event_chain_1_ak', answer_key=True)
create_pdf(data[:20], 'event_chain_1', answer_key=False)
create_pdf(data[:20], 'event_chain_1_triple', use_triples=True, answer_key=False)

This is pdfTeX, Version 3.14159265-2.6-1.40.16 (TeX Live 2015/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./event_chain_1_ak.tex
LaTeX2e <2016/02/01>
Babel <3.9q> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1enc.def))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/utf8.def
(/usr/share/texlive/texmf-dist/tex/latex/base/t1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/ot1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/omsenc.dfu)))
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty
(/usr/share/texlive/texmf-dist/tex/latex/bas

CalledProcessError: Command '['pdflatex', '--interaction=nonstopmode', 'event_chain_1_ak.tex']' returned non-zero exit status 1.

In [88]:
import os
checkpoint_path = os.path.join("gs://distorted-labeling-ml/logs/trans_3_16", "mask_rcnn_{}_*epoch*.h5".format(
            "test")).replace(
            "*epoch*", "{epoch:04d}")
os.path.join(os.path.abspath('./logs'), os.path.basename(checkpoint_path))

'/tf/mounted/cis700/logs/mask_rcnn_test_{epoch:04d}.h5'

In [89]:
log_dir = '/user_dir/logs/mask_rcnn_orange_{epoch:04d}.h5'
os.path.join(os.path.dirname(log_dir), '**.h5')

'/user_dir/logs/**.h5'